# Task: Predict the price of a house.

In [1]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import statistics
from statistics import mode
from statsmodels import robust

## Imports for Stepwise Regression
from dmba.featureSelection import stepwise_selection
from dmba import metric

no display found. Using non-interactive Agg backend


In [2]:
#loading the data
data = pd.read_csv('train.csv')

# Explanatory Data Analysis

In [3]:
#checking for the dimension of the dataset
data.shape

(1460, 81)

In [4]:
# the counts of the types of features
data.dtypes.value_counts()

object     43
int64      35
float64     3
dtype: int64

In [5]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
# Setting target column
target = data["SalePrice"]

# Dropping 'Id' and 'SalePrice' columns from data
data.drop(["Id","SalePrice"], inplace=True, axis=1)

In [41]:
# Viewing some basic statistical details like percentile, mean, std etc, using Pandas describe() function
data.describe().T

,count,mean,std,min,25%,50%,75%,max
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.000000,70.00,190.0
LotFrontage,1460.0,70.049958,22.024023,21.0,60.00,70.049958,79.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.500000,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.000000,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.000000,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.000000,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.000000,2004.00,2010.0
MasVnrArea,1460.0,103.685262,180.569112,0.0,0.00,0.000000,164.25,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.500000,712.25,5644.0
BsmtFinSF2,1460.0,46.549315,161.319273,0.0,0.00,0.000000,0.00,1474.0


Obsereved that Pandas describe() function just returns back the statistical details for numeric features. 

It can be seen that there is a big difference between the mean value and std for the numerical features such as  'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal'. Also the minimum value, 25%, 50%, 75% quartiles for these features are zero. Since majority of the values are 0, then it might not be much value to determine the sales price.

TR: "We can conclude that the observations for these features are outliers."

In [8]:
data['EnclosedPorch'].plot(kind='hist')

<AxesSubplot:ylabel='Frequency'>

In [43]:
plt.scatter(data['EnclosedPorch'], target)

# Finding the missing values

Missing values are the most common issue with any data science project. 
Most of the machine learning algorithms are not able to handle missing values. Thus, these missing values must be addressed before applying any machine learning algorithm. 

Missing values can be handled in different ways depending on if the missing values are continuous or categorical. So we first separet the numeric and categorical features. Then we try to count the number of missing values in each columns. 

It would be better to sort the columns in descending order in order to see the columns with highest missing values.

In [47]:
#extracting numeric and categorical columns
# data_numeric = Data.select_dtypes(include='number')
# data_categorical= Data.select_dtypes(include='object')
data_numeric = data.select_dtypes(include='number').columns.to_list()
data_categorical = data.select_dtypes(include='object').columns.to_list()

In [11]:
#getting numerical columns with the missing values in descending order.
data[data_numeric].isnull().sum().sort_values(ascending=False)/data.shape[0]

LotFrontage      0.177397
GarageYrBlt      0.055479
MasVnrArea       0.005479
WoodDeckSF       0.000000
BedroomAbvGr     0.000000
KitchenAbvGr     0.000000
TotRmsAbvGrd     0.000000
Fireplaces       0.000000
GarageCars       0.000000
GarageArea       0.000000
MSSubClass       0.000000
HalfBath         0.000000
EnclosedPorch    0.000000
3SsnPorch        0.000000
ScreenPorch      0.000000
PoolArea         0.000000
MiscVal          0.000000
MoSold           0.000000
OpenPorchSF      0.000000
FullBath         0.000000
BsmtHalfBath     0.000000
BsmtFullBath     0.000000
GrLivArea        0.000000
LowQualFinSF     0.000000
2ndFlrSF         0.000000
1stFlrSF         0.000000
TotalBsmtSF      0.000000
BsmtUnfSF        0.000000
BsmtFinSF2       0.000000
BsmtFinSF1       0.000000
YearRemodAdd     0.000000
YearBuilt        0.000000
OverallCond      0.000000
OverallQual      0.000000
LotArea          0.000000
YrSold           0.000000
dtype: float64

It is seen that three features "LotFrontage", "GarageYrBlt", and "MasVnrArea" have nearrly 18%, 5.5%, and .55% missing values, respectively. These missing values might be better to be replaced by the value of mean or median etc since it is only a handful.

In [12]:
#Viewing the basic statistical details of Categorical features using Pandas describe() function.
data[data_categorical].describe()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,...,1379,1379,1379,1379,1460,7,281,54,1460,1460
unique,5,2,2,4,4,2,5,3,25,9,...,6,3,5,5,3,3,4,4,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal
freq,1151,1454,50,925,1311,1459,1052,1382,225,1260,...,870,605,1311,1326,1340,3,157,49,1267,1198


In [13]:
#getting categorical columns with the missing values in descending order.
data[data_categorical].isnull().sum().sort_values(ascending=False)/data.shape[0]

PoolQC           0.995205
MiscFeature      0.963014
Alley            0.937671
Fence            0.807534
FireplaceQu      0.472603
GarageType       0.055479
GarageCond       0.055479
GarageQual       0.055479
GarageFinish     0.055479
BsmtFinType2     0.026027
BsmtExposure     0.026027
BsmtFinType1     0.025342
BsmtQual         0.025342
BsmtCond         0.025342
MasVnrType       0.005479
Electrical       0.000685
Functional       0.000000
KitchenQual      0.000000
CentralAir       0.000000
HeatingQC        0.000000
Heating          0.000000
PavedDrive       0.000000
SaleType         0.000000
MSZoning         0.000000
Street           0.000000
Condition2       0.000000
LotShape         0.000000
LandContour      0.000000
Utilities        0.000000
LotConfig        0.000000
LandSlope        0.000000
Neighborhood     0.000000
Condition1       0.000000
BldgType         0.000000
Foundation       0.000000
HouseStyle       0.000000
RoofStyle        0.000000
RoofMatl         0.000000
Exterior1st 

Columns like PoolQC, MiscFeature, Alley, Fence, have extremly large missing values (99%, 96%, 94%, 81% respectively). These columns might be better to remove from the dataset, since they are not informative. 

# Dealing with missing values

1- Deleting the column with missing data

2- Filling the Missing Values – Imputation:
  
     * Filling the missing data with the mean or median value, median absolute deviation etc if         it’s a numerical variable.
     * Filling the missing data with mode if it’s a categorical value.
     **???????Filling the numerical value with 0 or -999, or some other number that will not      occur in the data. This can be done so that the machine can recognize that the data 
     is not real or is different.


In [14]:
# Filling the missing values in numeric columns with mean

rows = data.shape[0]

## Looping through numeric columns
for col in data_numeric:
    
    ## Checking to 
    if data[col].isnull().sum() > (0.25 * rows):
        
        ## Dropping column from dataframe
        data.drop(col, axis=1, inplace=True)
        
        ## Dropping column from list of numeric columns
        data_numeric.remove(col)
    else:     
        replacement_value = data[col].mean()
        data[col].fillna(replacement_value, inplace=True ) 
data[data_numeric].isnull().sum().sum() 

0

In [15]:
#filling the missing values in categorical columns with mode

for col in data_categorical:
    if data[col].isnull().sum() > (0.25 * rows):
        data.drop(col, axis=1, inplace=True)
        data_categorical.remove(col)
    else:
        dropped = data[col].dropna()
        data[col] = data[col].fillna(mode(dropped)) 

data[data_categorical].isnull().sum().sum()    

1260

# Data visualization 

In [39]:
plt.hist(target)

(array([148., 723., 373., 135.,  51.,  19.,   4.,   3.,   2.,   2.]),
 array([ 34900., 106910., 178920., 250930., 322940., 394950., 466960.,
        538970., 610980., 682990., 755000.]),
 <BarContainer object of 10 artists>)

## Scatter plot

Scatter plot is a graph in which the values of two variables are plotted along two axes. It is a most basic type of plot that helps you visualize the relationship between two variables

In [16]:
fig = plt.figure(figsize = (14, 56)) 
for i in range(len(data_numeric)): 
    plt.subplot(6,6,i+1)   
    plt.title(data_numeric[i], size = 10, fontname = 'monospace')   
    a = plt.scatter(x=data[data_numeric[i]], y=target)  
    plt.ylabel('Saleprice')   
    plt.xlabel('')   
    plt.xticks(fontname = 'monospace')   
    plt.yticks([])   

fig.tight_layout(h_pad = 3) 
plt.show()

/var/folders/3h/n7g9d8x521gg4nfvf31y3_840000gn/T/ipykernel_47356/3826756400.py:12: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Histogram

In [ ]:
length=len(data_numeric)
fig = plt.figure(figsize = (14, 56)) 
      
for i in range(length): 
    plt.subplot(12,3,i+1)   
    sns.histplot(data=data, x=data_numeric[i])     

## Boxplot

A box plot is a statistical representation of the distribution of a variable through its quartiles. The ends of the box represent the lower and upper quartiles, while the median (second quartile) is marked by a line inside the box.

In [18]:
length=len(data_categorical)
fig = plt.figure(figsize = (14, 56)) 
      
for i in range(length):
    plt.subplot(9,5,i+1)   
    sns.boxplot(data=data, x=data_categorical[i], y=target)     

# Data Processing

In [19]:
# Dummifying the categorical
dummy_data = pd.get_dummies(data, drop_first=True)

In [70]:
# Splitting the data into train and test
X_train, X_valid, y_train, y_valid = train_test_split(dummy_data, target, test_size=0.30, random_state=0) 

# Baseline Model

In [71]:
# Fitting a linear regression
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [72]:
# Calculating train mse
train_preds = reg.predict(X_train)
train_mse = mean_squared_error(train_preds, y_train)
train_r2 = r2_score(train_preds, y_train)

In [73]:
# Calculating valid mse
prediction = reg.predict(X_valid)
valid_mse = mean_squared_error(prediction, y_valid)
valid_r2 = r2_score(prediction, y_valid)

In [74]:
# Printing train and valid mse
print('---MSE---')
print(f'Train MSE: {train_mse}')
print(f'Valid MSE: {valid_mse}')
print('---R^2---')
print(f'Train R^2: {train_r2}')
print(f'Valid R^2: {valid_r2}')

---MSE---
Train MSE: 325688756.1037614
Valid MSE: 2285602286.8297067
---R^2---
Train MSE: 0.9435945702487162
Valid MSE: 0.6981797528554989


I obseved a huge MSE (e40) I checked predictions and saw - values for predictions. Therefore i need to log prices 

In [66]:
## Need to log prices because the regression is predicting negative numbers
# np.log(target)
# np.exp(prediction)

array([ 290368.15993252,  140934.01602928,  126629.08087082,
        229810.41453626,  100248.32650795,   59910.63480001,
        249741.4631671 ,  131949.49965632,  773429.05334271,
        160453.02360947,  198227.71579679,  144805.02654001,
        224126.61052359,  122826.07413891,  110793.44041251,
        140532.60072622,  226624.70732162,  125228.27782192,
        139307.96508257,  166664.4398814 ,  120130.87839219,
        197611.01996241,   97833.3312183 ,  153250.67145275,
        194169.25801965,  153250.80401705,  169173.95363341,
         75111.08868643,  297791.17489078,  116310.72936948,
        154650.32436779,  197585.29116823,  154062.36184154,
        288539.70443777,  352168.89395101,  197104.55020246,
        274084.95574007,  124230.98562134,  237059.26193036,
        329103.06162   ,  198998.07441887,  122366.70049365,
        187344.7184079 ,  306406.45150626,  394081.4648805 ,
        148644.8588501 ,  115408.30803802,  137553.83898996,
        168571.3570439 ,

The MSEs look high for both the train and valid results. Based on the R^2, the variance is extremely high. The bias for training might be misleading because the R^2 metric gets higher for every additional feature added, even though it might not have any actual affect.

Need to make the model more stronger but at the same time make it less complex by reducing the number of dimensions.

## Scaling the numerical data

In [26]:
# Imports
from sklearn.preprocessing import MinMaxScaler

# Instantiating (creating) scaling MinMaxScaler object
scaler = MinMaxScaler()

# Fitting and transforming the numerical features for X_train
X_train[data_numeric] = scaler.fit_transform(X_train[data_numeric])

# Transforming the numerical features for X_test
X_valid[data_numeric] = scaler.transform(X_valid[data_numeric])

In [27]:
reg = LinearRegression()
reg.fit(X_train, y_train)

train_preds = reg.predict(X_train)
train_mse = mean_squared_error(train_preds, y_train)
train_r2 = r2_score(train_preds, y_train)

prediction = reg.predict(X_valid)
valid_mse = mean_squared_error(prediction, y_valid)
valid_r2 = r2_score(prediction, y_valid)

# Printing train and valid mse
print('---MSE---')
print(f'Train MSE: {train_mse}')
print(f'Valid MSE: {valid_mse}')
print('---R^2---')
print(f'Train R^2: {train_r2}')
print(f'Valid R^2: {valid_r2}')

---MSE---
Train MSE: 325693539.79452056
Valid MSE: 1.1019705589196946e+30
---R^2---
Train MSE: 0.9435847436220683
Valid MSE: -0.004055428174277509


In [83]:
def regression_report(model, X_train, y_train, X_valid, y_valid):
    '''
    Print out regression report
    
        ---MSE---
        Train MSE: 325693539.79452056
        Valid MSE: 1.1019705589196946e+30
        ---R^2---
        Train MSE: 0.9435847436220683
        Valid MSE: -0.004055428174277509
    
    
    Parameters
    
    
    Return
        None
    '''
    
    print('Good Job, Elaheh!')


# Validating the model

# cross validation 

Splitting a dataset into training and testing set is an essential and basic task when comes to getting a machine learning model ready for training. To determine if our model is overfitting or not we need to test it on unseen data (Validation set).

If a given model does not perform well on the validation set then it’s gonna perform worse when dealing with real live data. This notion makes Cross-Validation probably one of the most important concepts of machine learning which ensures the stability of our model.

Cross-Validation is just a method that simply reserves a part of data from the dataset and uses it for testing the model(Validation set), and the remaining data other than the reserved one is used to train the model.

##Why we should use cross validation.

* It helps us with model evaluation finally determining the quality of the model.
* Crucial to determining if the model is generalizing well to data.
* To check if the model is overfitting or underfitting.
* Finally, it lets us choose the model which had the best performance.



# Implementing the K-Fold Cross-Validation

The dataset is split into ‘k’ number of subsets, k-1 subsets then are used to train the model and the last subset is kept as a validation set to test the model. Then the score of the model on each fold is averaged to evaluate the performance of the model.

In [75]:
#Importing required libraries
import pandas as pd
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score
 
#seting up the dataset
X = dummy_data
y = np.log(target)
 
#Implementing cross validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=0)

RMSE_score = []
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

    model = LinearRegression()
    model.fit(X_train, y_train)
    pred_values = model.predict(X_test)
     
    RMSE = np.sqrt(mean_squared_error(np.exp(pred_values) , np.exp(y_test)))
    RMSE_score.append(RMSE)
    
print(f'MSE for each fold: {RMSE_score}')
print(f'Avg MSE : {np.mean(RMSE_score):.4f}')
print(f'Standard Deviation: {np.std(RMSE_score):.4f}')

MSE for each fold: [173573.17786055486, 26238.627210668652, 50992.490293935945, 23081.17744412856, 20988.70599109682]
Avg MSE : 58974.8358
Standard Deviation: 58308.5894


In [76]:
target.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

# Stepwise regression for feature selection

In [ ]:
def train_model(variables): 
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(X_train[variables], y_train)
    return model

def score_model(model, variables): 
    if len(variables) == 0:
        return metric.AIC_score(target, [target.mean()] * len(target), model, df=1)
    return metric.AIC_score(y_train, model.predict(X_train[variables]), model)

best_model, best_variables = stepwise_selection(X_train.columns, train_model,
                                                score_model, verbose=True)

print(f'Intercept: {best_model.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(best_variables, best_model.coef_):
    print(f' {name}: {coef}')

In [ ]:
len(best_variables)

In [ ]:
X_train[best_variables]

In [ ]:
best_model

In [ ]:
best_model.fit(X_train, y_train)

train_preds = best_model.predict(X_train)
train_mse = mean_squared_error(train_preds, y_train)
train_r2 = r2_score(train_preds, y_train)

prediction = best_model.predict(X_valid)
valid_mse = mean_squared_error(prediction, y_valid)
valid_r2 = r2_score(prediction, y_valid)

# Printing train and valid mse
print('---MSE---')
print(f'Train MSE: {train_mse}')
print(f'Valid MSE: {valid_mse}')
print('---R^2---')
print(f'Train MSE: {train_r2}')
print(f'Valid MSE: {valid_r2}')

Ridge Regression
Lasso Regression

In [84]:
regression_report()

Good Job, ELaheh!
